In [1]:
import torch
import numpy as np
from arsf_envi_reader import envi_header
import shutil
import os
import json
import math
import affine
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from osgeo import gdal,ogr,osr
from scipy.optimize import curve_fit
from tqdm import tqdm
import multiprocess as mp
from numpy import trapz

In [2]:
torch.cuda.is_available()

True

In [3]:
in_header = envi_header.find_hdr_file(r"D:\wenqu\aviris\20190704\ang20190704t193319rfl\ang20190704t193319_rfl_v2v2_img")
header_data = envi_header.read_hdr_file(in_header)

In [4]:
header_data

OrderedDict([('description',
              'AVIRIS-NG Measured Radiances in uW nm-1 cm-2 sr-1'),
             ('samples', '776'),
             ('lines', '15697'),
             ('bands', '425'),
             ('header offset', '0'),
             ('file type', 'ENVI'),
             ('data type', '4'),
             ('interleave', 'bil'),
             ('byte order', '0'),
             ('map info',
              'UTM , 1 , 1 , 627438.226598 , 7491171.13788 , 4.7 , 4.7 , 5 , North , WGS-84 , units=Meters , rotation=-18.0000000'),
             ('crosstrack scatter file',
              '/home/jchapman/src/isat/ang/cal/data/20170125_via_ang20160925t182412_crf'),
             ('wavelength units', 'Nanometers'),
             ('flat field file',
              '/home/jchapman/src/isat/ang/cal/data/20180727_ang20180628_BLUSS_avg_rows300-340_ff'),
             ('wavelength file',
              '/home/jchapman/src/isat/ang/cal/data/20190111_ang20180325t061759_wavelength_fit_full.txt'),
             ('w

In [5]:
# Get wavelengths and convert to NumPy array
low_res_wavelengths = header_data['wavelength'].split(',')
low_res_wavelengths = [float(w) for w in low_res_wavelengths]
low_res_wavelengths = np.array(low_res_wavelengths)


low_res_fwhm = header_data['fwhm'].split(',')
low_res_fwhm = [float(w) for w in low_res_fwhm]
low_res_fwhm = np.array(low_res_fwhm)

In [6]:
len(low_res_wavelengths), len(low_res_fwhm)

(425, 425)

In [7]:
def gauss(x, f, w):
    sigma = f/(2 * np.sqrt(2 *np.log(2)))
    y = np.exp((-(x-w)**2) / (2 * sigma**2))
    return y

In [8]:
high_res_img = gdal.Open(r'D:\wenqu\site1b\uav\site1b_multi_or_radiance')
high_res_radiance = gdal.Open(r'D:\wenqu\site1b\uav\site1b_multi_or_radiance').ReadAsArray() 

high_res_radiance.shape

(273, 1022, 3256)

In [9]:
high_res_wavelength = [float(b.split(" ")[0]) for b in high_res_img.GetMetadata().values() if b != "nm"]
high_res_wavelength.sort()
high_res_wavelength = np.array(high_res_wavelength)

In [10]:
gaussian_weights = []
normaliza_fwhm = []
normaliza_wav = []
for n in range(len(low_res_fwhm)):
#     print(n)
    a = []
    for i in high_res_wavelength:
        x = gauss(i, low_res_fwhm[n], low_res_wavelengths[n])
        if x < 0.5:
            x = 0
            a.append(x)
        else:
            a.append(x)
            normaliza_fwhm.append(low_res_fwhm[n])
            normaliza_wav.append(low_res_wavelengths[n])
#     print(a)
    gaussian_weights.append(a)

In [11]:
len(gaussian_weights),gaussian_weights

(425,
 [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0

In [12]:
normaliza_fwhm

[5.590000000000001,
 5.590000000000001,
 5.590000000000001,
 5.590000000000001,
 5.590000000000001,
 5.6,
 5.6,
 5.6,
 5.6,
 5.6,
 5.6,
 5.6,
 5.6,
 5.6,
 5.6,
 5.61,
 5.61,
 5.61,
 5.61,
 5.61,
 5.61,
 5.61,
 5.61,
 5.62,
 5.62,
 5.62,
 5.62,
 5.62,
 5.62,
 5.62,
 5.62,
 5.62,
 5.62,
 5.62,
 5.63,
 5.63,
 5.63,
 5.63,
 5.63,
 5.63,
 5.63,
 5.63,
 5.64,
 5.64,
 5.64,
 5.64,
 5.64,
 5.64,
 5.64,
 5.64,
 5.64,
 5.64,
 5.6499999999999995,
 5.6499999999999995,
 5.6499999999999995,
 5.6499999999999995,
 5.6499999999999995,
 5.6499999999999995,
 5.6499999999999995,
 5.6499999999999995,
 5.6499999999999995,
 5.6499999999999995,
 5.66,
 5.66,
 5.66,
 5.66,
 5.66,
 5.66,
 5.66,
 5.66,
 5.66,
 5.66,
 5.66,
 5.66,
 5.67,
 5.67,
 5.67,
 5.67,
 5.67,
 5.67,
 5.67,
 5.67,
 5.67,
 5.67,
 5.67,
 5.68,
 5.68,
 5.68,
 5.68,
 5.68,
 5.68,
 5.68,
 5.68,
 5.68,
 5.68,
 5.68,
 5.68,
 5.68,
 5.68,
 5.6899999999999995,
 5.6899999999999995,
 5.6899999999999995,
 5.6899999999999995,
 5.6899999999999995,
 5.6899

In [13]:
len(normaliza_wav), normaliza_wav

(311,
 [396.749576,
  401.759576,
  401.759576,
  406.76957600000003,
  406.76957600000003,
  411.77957599999996,
  411.77957599999996,
  411.77957599999996,
  416.789576,
  416.789576,
  416.789576,
  421.799576,
  421.799576,
  426.80957600000005,
  426.80957600000005,
  431.819576,
  431.819576,
  431.819576,
  436.819576,
  436.819576,
  436.819576,
  441.829576,
  441.829576,
  446.839576,
  446.839576,
  446.839576,
  451.84957599999996,
  451.84957599999996,
  451.84957599999996,
  456.859576,
  456.859576,
  456.859576,
  461.869576,
  461.869576,
  466.87957600000004,
  466.87957600000004,
  466.87957600000004,
  471.87957600000004,
  471.87957600000004,
  471.87957600000004,
  476.889576,
  476.889576,
  481.899576,
  481.899576,
  486.909576,
  486.909576,
  486.909576,
  491.919576,
  491.919576,
  491.919576,
  496.929576,
  496.929576,
  501.93957600000005,
  501.93957600000005,
  506.94957600000004,
  506.94957600000004,
  506.94957600000004,
  511.94957600000004,
  511.

In [14]:
high_res_img_flatten = high_res_radiance.reshape(273, -1)
high_res_img_flatten.shape

(273, 3327632)

In [15]:
import pandas as pd
high_res_irradiance = np.array(pd.read_csv('D:\wenqu\site1b\white_panel_radiance.csv')['irradiance'])
high_res_irradiance.shape

(273,)

In [16]:
from tqdm import tqdm

In [17]:
pixel_all = []

denoms = []
for j in range(len(gaussian_weights)):
    denoms.append(trapz(gaussian_weights[j]*high_res_irradiance, high_res_wavelength))
denoms = np.array(denoms)

In [18]:
# move numpy array from cpu to gpu
denoms_tensor = torch.from_numpy(denoms).cuda()
gaussian_weights_tensor = torch.from_numpy(np.array(gaussian_weights)).cuda()

high_res_img_flatten_tensor = torch.from_numpy(high_res_img_flatten)
high_res_wavelength_tensor = torch.from_numpy(high_res_wavelength[None,]).cuda().repeat(len(gaussian_weights), 1)

In [19]:
batch_pixels = []
if os.path.exists('pixel_all'): 
    shutil.rmtree('pixel_all')
os.makedirs('pixel_all')
with torch.no_grad():
    for i in tqdm(range(high_res_img_flatten.shape[1])):
        pixel_single = []
        # I have 425 guassian function, one single pixel has 273 bands, I need to feed all these 273 bands to each guassian function
        # this step is to repeat 273 bands for 425 times to multiply with guassian function for one time, therefore, I will have 425
        #momin results, each result will include 273 values. So the shape for nomin_inputs is 425x273
        nomin_inputs = gaussian_weights_tensor * high_res_img_flatten_tensor[:, i][None, ].cuda().repeat(gaussian_weights_tensor.shape[0], 1)    #print(nomin_inputs, nomin_inputs.shape, type(nomin_inputs))
        # there are 425 nomin_inputs, we need to do 425 times integral, repeat 425 times
        nomins = torch.trapz(nomin_inputs, high_res_wavelength_tensor)
        pixel_single = nomins / denoms_tensor
        batch_pixels.append(pixel_single.detach().cpu().numpy())
        
        if i % 1000000 == 999999:
            batch_pixels = np.array(batch_pixels)
            np.save(f'pixel_all/{i:10d}.npy', batch_pixels) 
            batch_pixels = []
if len(batch_pixels):
    batch_pixels = np.array(batch_pixels)
    np.save(f'pixel_all/{i:10d}.npy', batch_pixels)
    batch_pixels = []  
        
# pixel_all = np.array(pixel_all)

100%|██████████████████████████████████████████████████████████████████████| 3327632/3327632 [17:19<00:00, 3200.92it/s]
